In [1]:
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
    
# Path to your image directory
# image_directory='/kaggle/input/gallbladder/Gallblader Diseases Dataset/1Gallstones'

# Initialize variables to collect data

# Function to explore images
def explore_images(image_directory):
    image_sizes = []
    image_channels = []
    pixel_values = []

#     print(image_directory)
    image_paths = [os.path.join(image_directory, fname) for fname in os.listdir(image_directory) if fname.endswith(('jpg', 'png', 'jpeg'))]
#     print(image_paths)
    for image_path in image_paths:
        # Open the image
        image = Image.open(image_path)
        
        # Get image size (width, height)
        image_sizes.append(image.size)
        
        # Get image color channels (1 for grayscale, 3 for RGB)
        image_channels.append(len(image.getbands()))  # 'RGB' gives 3 channels, 'L' gives 1
        
        # Convert image to numpy array for statistical analysis
        image_array = np.array(image)
        pixel_values.append(image_array)

    # Return the collected statistics
    image_sizes, image_channels, pixel_values


    # Find unique image sizes
    unique_image_sizes = set(image_sizes)  # Using set to get unique sizes

    # Print the number of unique image sizes
    print(f"Number of unique image sizes: {len(unique_image_sizes)}")
    print(f"Unique image sizes: {unique_image_sizes}")
# # Statistics about image sizes
    image_sizes = np.array(image_sizes)
    image_channels = np.array(image_channels)

    # Print basic statistics about image sizes
    print("Image Dimensions (width x height):")
    print(f"Minimum size: {image_sizes.min(axis=0)}")
    print(f"Maximum size: {image_sizes.max(axis=0)}")
    print(f"Average size: {image_sizes.mean(axis=0)}")

    # Check if images are grayscale or RGB
    print(f"\nNumber of Grayscale images: {np.sum(image_channels == 1)}")
    print(f"Number of RGB images: {np.sum(image_channels == 3)}")





In [2]:
import os
root_directory = '/kaggle/input/gallbladder/Gallblader Diseases Dataset'  # Update to your dataset path
directory=os.listdir('/kaggle/input/gallbladder/Gallblader Diseases Dataset')
# print(directory)
for d in directory:
    print(d)
    explore_images(os.path.join(root_directory,d))
    print('\n')
    print('**************************************************')

8Carcinoma
Number of unique image sizes: 2
Unique image sizes: {(2400, 1800), (1200, 900)}
Image Dimensions (width x height):
Minimum size: [1200  900]
Maximum size: [2400 1800]
Average size: [1200.75471698  900.56603774]

Number of Grayscale images: 0
Number of RGB images: 1590


**************************************************
6Polyps and cholesterol crystals
Number of unique image sizes: 2
Unique image sizes: {(900, 1200), (1200, 900)}
Image Dimensions (width x height):
Minimum size: [900 900]
Maximum size: [1200 1200]
Average size: [1193.52941176  906.47058824]

Number of Grayscale images: 0
Number of RGB images: 1020


**************************************************
3cholecystitis
Number of unique image sizes: 3
Unique image sizes: {(2400, 1800), (900, 1200), (1200, 900)}
Image Dimensions (width x height):
Minimum size: [900 900]
Maximum size: [2400 1800]
Average size: [1194.76439791  907.06806283]

Number of Grayscale images: 0
Number of RGB images: 1146


******************

#  Google Vit

In [3]:
pip install tensorflow tensorflow-hub tensorflow-datasets

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pillow

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Loading Libraries
import os
import tensorflow as tf
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model
import tensorflow_hub as hub

In [6]:
root_directory='/kaggle/input/gallbladder/Gallblader Diseases Dataset'

train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2  # Split 20% for validation (testing)
)

train_generator= train_datagen.flow_from_directory(
    root_directory,
    target_size=(384,384),
    batch_size=32,
    class_mode='categorical',
    subset='training')

val_generator= train_datagen.flow_from_directory(
    root_directory,
    target_size=(384,384),
    batch_size=32,
    class_mode='categorical',
    subset='validation')
    

print(f"Train data: {train_generator.samples} images")
print(f"Validation data: {val_generator.samples} images")
    
    
    
    
    

    
    
    
    
    

Found 8556 images belonging to 9 classes.
Found 2136 images belonging to 9 classes.
Train data: 8556 images
Validation data: 2136 images


In [7]:
pip install transformers torch torchvision


/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


Note: you may need to restart the kernel to use updated packages.


In [8]:
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from transformers import AutoImageProcessor, AutoModelForImageClassification

# Load processor and model
model_name = "google/vit-base-patch16-384"
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

# Custom Dataset Class
class CustomImageDataset(Dataset):
    def __init__(self, image_paths, labels, processor):
        self.image_paths = image_paths
        self.labels = labels
        self.processor = processor

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        inputs = self.processor(images=image, return_tensors="pt")
        image_tensor = inputs['pixel_values'].squeeze(0)
        label = self.labels[idx]
        return image_tensor, label

# Load image paths and labels
root_directory = "/kaggle/input/gallbladder/Gallblader Diseases Dataset"
class_names = sorted(os.listdir(root_directory))
image_paths = []
labels = []

for label, class_name in enumerate(class_names):
    class_dir = os.path.join(root_directory, class_name)
    for img_name in os.listdir(class_dir):
        image_paths.append(os.path.join(class_dir, img_name))
        labels.append(label)

# Stratified split for train and test sets
train_paths, test_paths, train_labels, test_labels = train_test_split(
    image_paths, labels, test_size=0.2, stratify=labels, random_state=42
)

# Create train and test datasets
train_dataset = CustomImageDataset(train_paths, train_labels, processor)
test_dataset = CustomImageDataset(test_paths, test_labels, processor)

# Create DataLoaders for training and testing
batch_size = 8
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Check the class distribution in train and test sets
from collections import Counter

train_class_counts = Counter(train_labels)
test_class_counts = Counter(test_labels)
print("Training class distribution:", train_class_counts)
print("Testing class distribution:", test_class_counts)


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/69.7k [00:00<?, ?B/s]

Fast image processor class <class 'transformers.models.vit.image_processing_vit_fast.ViTImageProcessorFast'> is available for this model. Using slow image processor class. To use the fast image processor class set `use_fast=True`.


model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

Training class distribution: Counter({7: 1272, 0: 1061, 3: 979, 1: 936, 6: 931, 2: 917, 4: 849, 5: 816, 8: 792})
Testing class distribution: Counter({7: 318, 0: 265, 3: 245, 1: 234, 6: 233, 2: 229, 4: 213, 5: 204, 8: 198})


In [9]:
import torch
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from tqdm import tqdm
from sklearn.metrics import accuracy_score

# Set device to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Training hyperparameters
epochs = 5
learning_rate = 1e-4
optimizer = Adam(model.parameters(), lr=learning_rate)
criterion = CrossEntropyLoss()

# Training loop
def train_model(model, train_loader):
    model.train()  # Set the model to training mode
    total_loss = 0
    all_preds = []
    all_labels = []
    
    for images, labels in tqdm(train_loader, desc="Training"):
        images, labels = images.to(device), labels.to(device)
        
        # Forward pass
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Track loss and accuracy
        total_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    epoch_loss = total_loss / len(train_loader)
    epoch_accuracy = accuracy_score(all_labels, all_preds)
    
    print(f"Training Loss: {epoch_loss:.4f}, Training Accuracy: {epoch_accuracy:.4f}")

# Evaluation loop
def evaluate_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(test_loader, desc="Evaluating"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images).logits
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    test_accuracy = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {test_accuracy:.4f}")
    return test_accuracy

# Training the model across epochs and evaluating after each epoch
best_accuracy = 0.0
for epoch in range(epochs):
    print(f"\nEpoch {epoch + 1}/{epochs}")
    train_model(model, train_loader)
    test_accuracy = evaluate_model(model, test_loader)
    
    # Save model if it has the best accuracy
    if test_accuracy > best_accuracy:
        best_accuracy = test_accuracy
        torch.save(model.state_dict(), "best_vit_model.pth")
        print("Best model saved with accuracy:", best_accuracy)

print("Training complete. Best test accuracy:", best_accuracy)



Epoch 1/5


Training: 100%|██████████| 1070/1070 [12:14<00:00,  1.46it/s]


Training Loss: 0.8635, Training Accuracy: 0.7107


Evaluating: 100%|██████████| 268/268 [01:29<00:00,  3.01it/s]


Test Accuracy: 0.9444
Best model saved with accuracy: 0.9443665264142123

Epoch 2/5


...

Test Accuracy: 0.9906
Best model saved with accuracy: 0.9906498363721365

Epoch 5/5


Training: 100%|██████████| 1070/1070 [12:11<00:00,  1.46it/s]


Training Loss: 0.0411, Training Accuracy: 0.9869


Evaluating: 100%|██████████| 268/268 [01:27<00:00,  3.07it/s]

Test Accuracy: 0.9705
Training complete. Best test accuracy: 0.9906498363721365
